In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import statsmodels.api as sm
%matplotlib inline
import seaborn as sns; sns.set()

# pd.get_option("display.max_columns")
# pd.set_option('display.max_columns', 100)

参考資料：<br>
https://run.unl.pt/bitstream/10362/33864/1/TGI0135.pdf<br>

In [2]:
DATA_DIR = '/Users/sasanoshouta/Desktop/DIVE_INTO_CODE/kaggle/MDataFiles_Stage2'
STAGE_1 = True

# シード情報特徴量・qualityの作成

In [3]:
regular_results = pd.read_csv(DATA_DIR+'/MRegularSeasonCompactResults.csv')

In [4]:
def prepare_data(df):
    dfswap = df[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT']]

    dfswap.loc[df['WLoc'] == 'H', 'WLoc'] = 'A'
    dfswap.loc[df['WLoc'] == 'A', 'WLoc'] = 'H'
    df.columns.values[6] = 'location'
    dfswap.columns.values[6] = 'location'         
    df.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(df.columns)]
    dfswap.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(dfswap.columns)]
    output = pd.concat([df, dfswap]).sort_index().reset_index(drop=True)
    
    return output

regular_results = prepare_data(regular_results)
# regular_results = regular_results.loc[:, ['Season', 'T1_TeamID', 'T2_TeamID', 'location', 'NumOT']]
# regular_results = regular_results.loc[regular_results['Season'] >= 2015]
regular_results['T1_win'] = 0
regular_results['T2_win'] = 0
regular_results.loc[regular_results['T1_Score'] > regular_results['T2_Score'], 'T1_win'] = 1
regular_results.loc[regular_results['T1_Score'] < regular_results['T2_Score'], 'T2_win'] = 1
regular_results

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_win,T2_win
0,1985,20,1228,81,1328,64,N,0,1,0
1,1985,20,1328,64,1228,81,N,0,0,1
2,1985,25,1354,70,1106,77,A,0,0,1
3,1985,25,1106,77,1354,70,H,0,1,0
4,1985,25,1112,63,1223,56,H,0,1,0
...,...,...,...,...,...,...,...,...,...,...
341465,2021,132,1104,80,1261,79,N,0,1,0
341466,2021,132,1153,54,1222,91,N,0,0,1
341467,2021,132,1222,91,1153,54,N,0,1,0
341468,2021,132,1228,91,1326,88,N,1,1,0


In [5]:
# convert to str, so the model would treat TeamID them as factors
regular_results['T1_TeamID'] = regular_results['T1_TeamID'].astype(str)
regular_results['T2_TeamID'] = regular_results['T2_TeamID'].astype(str)

# make it a binary task
regular_results['win'] = np.where(regular_results['T1_Score']>regular_results['T2_Score'], 1, 0)

def team_quality(season):
    """
    Calculate team quality for each season seperately. 
    Team strength changes from season to season (students playing change!)
    So pooling everything would be bad approach!
    """
    formula = 'win~-1+T1_TeamID+T2_TeamID'
    glm = sm.GLM.from_formula(formula=formula, 
                              data=regular_results.loc[regular_results.Season==season,:], 
                              family=sm.families.Binomial()).fit()
    
    # extracting parameters from glm
    quality = pd.DataFrame(glm.params).reset_index()
    quality.columns = ['TeamID','beta']
    quality['Season'] = season
    # taking exp due to binomial model being used
    quality['quality'] = np.exp(quality['beta'])
    # only interested in glm parameters with T1_, as T2_ should be mirroring T1_ ones
    quality = quality.loc[quality.TeamID.str.contains('T1_')].reset_index(drop=True)
    quality['TeamID'] = quality['TeamID'].apply(lambda x: x[10:14]).astype(int)
    return quality

In [6]:
%%time
team_quality = pd.concat([team_quality(1985),
                          team_quality(1986),
                          team_quality(1987),
                          team_quality(1988),
                          team_quality(1989),
                          team_quality(1990),
                          team_quality(1991),
                          team_quality(1992),
                          team_quality(1993),
                          team_quality(1994),
                          team_quality(1995),
                          team_quality(1996),
                          team_quality(1997),
                          team_quality(1998),
                          team_quality(1999),
                          team_quality(2000),
                          team_quality(2001),
                          team_quality(2002),
                          team_quality(2003),
                          team_quality(2004),
                          team_quality(2005),
                          team_quality(2006),
                          team_quality(2007),
                          team_quality(2008),
                          team_quality(2009),
                          team_quality(2010),
                          team_quality(2011),
                          team_quality(2012),
                          team_quality(2013),
                          team_quality(2014),
                          team_quality(2015),
                          team_quality(2016),
                          team_quality(2017),
                          team_quality(2018),
                          team_quality(2019),
                          team_quality(2020),
                          team_quality(2021)
                          ]).reset_index(drop=True)

team_quality

CPU times: user 15min 12s, sys: 57.3 s, total: 16min 9s
Wall time: 10min 17s


,TeamID,beta,Season,quality
0,1102,-1.180128e-14,1985,1.000000
1,1103,2.135582e-01,1985,1.238076
2,1104,4.099354e+00,1985,60.301301
3,1106,-7.325023e-01,1985,0.480705
4,1108,1.477263e+00,1985,4.380937
...,...,...,...,...
11936,1467,-8.085556e-01,2021,0.445501
11937,1468,-8.643418e-01,2021,0.421329
11938,1469,-2.881844e+00,2021,0.056031
11939,1470,-2.525066e+00,2021,0.080053


In [7]:
# 各シードに配置された時の勝率、パワーランク作成
tourney_results = pd.read_csv(DATA_DIR+'/MNCAATourneyCompactResults.csv')
seeds = pd.read_csv(DATA_DIR+'/MNCAATourneySeeds.csv')

tourney_results = prepare_data(tourney_results)

team_quality_copy = team_quality.copy()

team_quality_T1 = team_quality_copy[['TeamID','Season','quality']]
team_quality_T1.columns = ['T1_TeamID','Season','T1_quality']
team_quality_T2 = team_quality_copy[['TeamID','Season','quality']]
team_quality_T2.columns = ['T2_TeamID','Season','T2_quality']

tourney_results['T1_TeamID'] = tourney_results['T1_TeamID'].astype(int)
tourney_results['T2_TeamID'] = tourney_results['T2_TeamID'].astype(int)
tourney_results = tourney_results.merge(team_quality_T1, on = ['T1_TeamID','Season'], how = 'left')
tourney_results = tourney_results.merge(team_quality_T2, on = ['T2_TeamID','Season'], how = 'left')

tourney_results = tourney_results.loc[tourney_results['DayNum'] >= 136].reset_index(drop=True)
tourney_results = tourney_results.loc[tourney_results['Season'] < 2015].reset_index(drop=True)

seeds['seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds['division'] = seeds['Seed'].apply(lambda x: x[0])

seeds_T1 = seeds[['Season','TeamID','seed','division']].copy()
seeds_T2 = seeds[['Season','TeamID','seed','division']].copy()
seeds_T1.columns = ['Season','T1_TeamID','T1_seed','T1_division']
seeds_T2.columns = ['Season','T2_TeamID','T2_seed','T2_division']

tourney_results = tourney_results.merge(seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_results = tourney_results.merge(seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')

tourney_results['T1_powerrank'] = tourney_results.groupby(['Season','T1_division'])['T1_quality'].rank(method='dense', ascending=False).astype(int)
tourney_results['T2_powerrank'] = tourney_results.groupby(['Season','T2_division'])['T2_quality'].rank(method='dense', ascending=False).astype(int)

tourney_results['win'] = np.where(tourney_results['T1_Score'] > tourney_results['T2_Score'], 1, 0)

mean_win_ratio = pd.DataFrame({'seed_win_ratio': tourney_results.groupby('T1_seed')['win'].mean(),
                               'powerrank_win_ratio': tourney_results.groupby('T1_powerrank')['win'].mean()})
mean_win_ratio = mean_win_ratio.reset_index().rename(columns={'index':'seed'})
mean_win_ratio

,seed,seed_win_ratio,powerrank_win_ratio
0,1,0.796407,0.788187
1,2,0.712871,0.701058
2,3,0.654762,0.664756
3,4,0.609836,0.578947
4,5,0.523810,0.547170
5,6,0.536965,0.524390
6,7,0.466368,0.512295
7,8,0.422330,0.441860
8,9,0.365079,0.400000
9,10,0.387755,0.371728


In [141]:
# シード情報とチームクオリティ、シード勝率の合体
TourneyCompactResults = pd.read_csv(DATA_DIR+'/MNCAATourneyCompactResults.csv')

A_t1 = TourneyCompactResults[TourneyCompactResults.WLoc == 'N']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "TeamID1"})

A_t2 = TourneyCompactResults[TourneyCompactResults.WLoc == 'N']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "TeamID2"})
A_t1.index = A_t1.index.rename(['Season', 'TeamID'])
A_t2.index = A_t2.index.rename(['Season', 'TeamID'])
A_t2
A = A_t1.join(A_t2, how='outer').reset_index().drop(['TeamID1', 'TeamID2'], axis=1)
del A_t1, A_t2

seeds['seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds['division'] = seeds['Seed'].apply(lambda x: x[0])

l_seed = A.merge(seeds, on=['Season', 'TeamID'], how='left').drop('Seed', axis=1)
# l_seed = l_seed.drop('location_N', axis=1) # locationがリークさせていた→シンプルにそのトーナメントでの試合数になってしまうから
# del A, seeds
# l_seed['division_rank'] = l_seed.division.replace({'W':4, 'X':3, 'Y':2, 'Z':1})
# l_seed['seed_ratio'] = l_seed['seed'] * l_seed['division_rank']
# l_seed.drop(['seed', 'division', 'division_rank'], inplace=True, axis=1)
q_seed = l_seed.merge(team_quality, on=['Season', 'TeamID'], how='left').drop(['beta'], axis=1)\
        .merge(mean_win_ratio, on='seed', how='left')
q_seed = q_seed.drop(['seed_win_ratio', 'powerrank_win_ratio'], axis=1)
q_seed

,Season,TeamID,seed,division,quality
0,1985,1104,7,X,60.301301
1,1985,1112,10,X,26.511951
2,1985,1116,9,X,40.730603
3,1985,1120,11,Z,45.880625
4,1985,1130,11,Y,46.265708
...,...,...,...,...,...
2281,2019,1439,4,W,25.587572
2282,2019,1449,9,Y,10.468700
2283,2019,1458,5,Z,26.839622
2284,2019,1459,7,Y,26.820409


# シード情報以外で回す場合はここから！

In [173]:
MRSCResults = pd.read_csv(DATA_DIR + '/MRegularSeasonCompactResults.csv')

A_w = MRSCResults[MRSCResults.WLoc == 'A']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_A"})
N_w = MRSCResults[MRSCResults.WLoc == 'N']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_N"})
H_w = MRSCResults[MRSCResults.WLoc == 'H']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_H"})
win = A_w.join(N_w, how='outer').join(H_w, how='outer').fillna(0)

H_l = MRSCResults[MRSCResults.WLoc == 'A']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "lost_H"})
N_l = MRSCResults[MRSCResults.WLoc == 'N']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "lost_N"})
A_l = MRSCResults[MRSCResults.WLoc == 'H']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "lost_A"})
lost = A_l.join(N_l, how='outer').join(H_l, how='outer').fillna(0)

win.index = win.index.rename(['Season', 'TeamID'])
lost.index = lost.index.rename(['Season', 'TeamID'])
wl = win.join(lost, how='outer').reset_index()
wl['win_pct_A'] = wl['win_A'] / (wl['win_A'] + wl['lost_A'])
wl['win_pct_N'] = wl['win_N'] / (wl['win_N'] + wl['lost_N'])
wl['win_pct_H'] = wl['win_H'] / (wl['win_H'] + wl['lost_H'])
wl['win_pct_All'] = (wl['win_A'] + wl['win_N'] + wl['win_H']) / \
    (wl['win_A'] + wl['win_N'] + wl['win_H'] + wl['lost_A']\
     + wl['lost_N'] + wl['lost_H'])

del A_w, N_w, H_w, H_l, N_l, A_l, win, lost

In [174]:
MRSCResults['relScore'] = MRSCResults.WScore - MRSCResults.LScore

w_scr = MRSCResults.loc[:, ['Season', 'WTeamID', 'WScore', 'WLoc','relScore']]
w_scr.columns = ['Season', 'TeamID','Score','Loc','relScore']
l_scr = MRSCResults.loc[:, ['Season', 'LTeamID', 'LScore', 'WLoc','relScore']]
l_scr['WLoc'] = l_scr.WLoc.apply(lambda x: 'H' if x == 'A' else 'A' \
                                 if x == 'H' else 'N')
l_scr['relScore'] = -1 * l_scr.relScore 
l_scr.columns = ['Season', 'TeamID','Score','Loc','relScore']
wl_scr = pd.concat([w_scr,l_scr])

A_scr = wl_scr[wl_scr.Loc == 'A'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_A", "relScore": "relScore_A"})
N_scr = wl_scr[wl_scr.Loc == 'N'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_N", "relScore": "relScore_N"})
H_scr = wl_scr[wl_scr.Loc == 'H'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_H", "relScore": "relScore_H"})
All_scr = wl_scr.groupby(['Season','TeamID'])['Score','relScore']\
    .mean().rename(columns={"Score": "Score_All", "relScore": "relScore_All"})
scr = A_scr.join(N_scr, how='outer').join(H_scr, how='outer')\
    .join(All_scr, how='outer').fillna(0).reset_index()

del w_scr, l_scr, wl_scr, A_scr, H_scr, N_scr, All_scr

<ipython-input-174-3cd860bc9e4b>:12: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  A_scr = wl_scr[wl_scr.Loc == 'A'].groupby(['Season','TeamID'])\
<ipython-input-174-3cd860bc9e4b>:15: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  N_scr = wl_scr[wl_scr.Loc == 'N'].groupby(['Season','TeamID'])\
<ipython-input-174-3cd860bc9e4b>:18: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  H_scr = wl_scr[wl_scr.Loc == 'H'].groupby(['Season','TeamID'])\
<ipython-input-174-3cd860bc9e4b>:21: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  All_scr = wl_scr.groupby(['Season','TeamID'])['Score','relScore']\


In [175]:
scr

,Season,TeamID,Score_A,relScore_A,Score_N,relScore_N,Score_H,relScore_H,Score_All,relScore_All
0,1985,1102,62.833333,-8.166667,55.50,-12.500000,64.900000,-1.600000,63.083333,-5.791667
1,1985,1103,60.800000,-8.300000,0.00,0.000000,61.230769,1.000000,61.043478,-3.043478
2,1985,1104,65.181818,4.000000,53.00,-0.333333,73.687500,11.937500,68.500000,7.800000
3,1985,1106,72.272727,-7.272727,64.50,-14.000000,74.000000,5.000000,71.625000,-3.791667
4,1985,1108,78.583333,1.583333,77.75,5.250000,91.222222,17.666667,83.000000,7.960000
...,...,...,...,...,...,...,...,...,...,...
11936,2021,1467,62.555556,-1.555556,0.00,0.000000,70.000000,0.777778,66.277778,-0.388889
11937,2021,1468,75.888889,7.444444,70.00,-3.000000,69.125000,1.625000,72.555556,4.277778
11938,2021,1469,72.222222,-12.333333,0.00,0.000000,63.500000,-9.000000,67.631579,-10.578947
11939,2021,1470,65.727273,-2.909091,0.00,0.000000,58.750000,-8.000000,63.866667,-4.266667


In [176]:
# ここでチーム毎の試合データをもとに特徴量作成をやっている(1試合毎のやつ)
# 攻撃回数=2P試投数 + 3P試投数 + ターンオーバー数 + FT試投数 × 0.43)を用いて算出した。
# そして、総得点数を攻撃回数で除することで攻撃効率(Points Per Possession)を求めた。

MRSDetailedResults = pd.read_csv(DATA_DIR + '/MRegularSeasonDetailedResults.csv')
# 対戦相手のスコアを保持
MRSDetailedResults['WO_PTS'] = MRSDetailedResults.LScore
MRSDetailedResults['LO_PTS'] = MRSDetailedResults.WScore
# 対戦相手のオフェンスリバウンド回数、ディフェンスリバウンド率を保持
MRSDetailedResults['WO_DREB'] = MRSDetailedResults.LDR
MRSDetailedResults['LO_DREB'] = MRSDetailedResults.WDR
MRSDetailedResults['WO_OREB'] = MRSDetailedResults.LOR
MRSDetailedResults['LO_OREB'] = MRSDetailedResults.WOR
MRSDetailedResults['WO_REB'] = MRSDetailedResults.LDR + MRSDetailedResults.LOR
MRSDetailedResults['LO_REB'] = MRSDetailedResults.WDR + MRSDetailedResults.WOR
#対戦相手のFGA
MRSDetailedResults['WO_FGA'] = MRSDetailedResults.LFGA
MRSDetailedResults['LO_FGA'] = MRSDetailedResults.WFGA
# 対戦相手のFTA
MRSDetailedResults['WO_FTA'] = MRSDetailedResults.LFTA
MRSDetailedResults['LO_FTA'] = MRSDetailedResults.WFTA
# 対戦相手のTO
MRSDetailedResults['WO_TO'] = MRSDetailedResults.LTO
MRSDetailedResults['LO_TO'] = MRSDetailedResults.WTO

w = MRSDetailedResults.loc[:, ['Season', 'WScore', 'WTeamID', 'WFGM','WFGA','WFGM3'
                               ,'WFGA3','WFTM','WFTA','WOR','WDR','WAst',
                               'WTO','WStl','WBlk','WPF', 'WO_PTS', 'WO_DREB', 'WO_OREB', 'WO_REB',
                              'WO_FGA', 'WO_FTA', 'WO_TO']]
w.columns = ['Season', 'Score', 'TeamID', 'FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR',
             'Ast','TO','Stl','Blk','PF', 'O_PTS', 'O_DREB', 'O_OREB', 'O_REB', 'O_FGA', 'O_FTA', 'O_TO']
l = MRSDetailedResults.loc[:, ['Season', 'LScore', 'LTeamID', 'LFGM','LFGA','LFGM3',
                               'LFGA3','LFTM','LFTA','LOR','LDR','LAst',
                               'LTO','LStl','LBlk','LPF', 'LO_PTS', 'LO_DREB', 'LO_OREB', 'LO_REB',
                              'LO_FGA', 'LO_FTA', 'LO_TO']]
l.columns = ['Season', 'Score', 'TeamID', 'FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR',
             'Ast','TO','Stl','Blk','PF', 'O_PTS', 'O_DREB', 'O_OREB', 'O_REB', 'O_FGA', 'O_FTA', 'O_TO']

detail = pd.concat([w,l])
detail['goal_rate'] = detail.FGM / detail.FGA 
detail['3p_goal_rate'] = detail.FGM3 / detail.FGA3  
detail['ft_goal_rate'] = detail.FTM  / detail.FTA 
detail['total_attack'] = detail.FGA + detail.TO + detail.FTA * 0.43
detail['Points_Per_Possession'] = detail.Score / detail.total_attack
# https://bstatsplus.com/glossary このサイト参考にスタッツを計算
# 参考サイトその2:https://www.nogawanogawa.com/entry/basketball_analysis
# POSS:ポゼッション。攻撃回数。
detail['POSS'] = detail.FGA + 0.44 * detail.FTA + detail.TO - detail.OR
# ORTG:オフェンスレーティング。攻撃力。100POSSあたりの得点。
detail['ORTG'] = detail.Score / detail.POSS * 100
# DRTG:ディフェンスレーティング。守備力。100POSSあたりの失点。
detail['DRTG'] = detail.O_PTS / detail.POSS * 100
# NRTG:ネットレーティング。総合力。攻撃力と守備力の差でみる真の力。
detail['NRTG'] = detail.ORTG - detail.DRTG
# eFG%:得点効率。Effective FG%。「Four Factors」の1つ。3Pの価値は2Pの1.5倍という重みを付けて効率を評価。50%を超えるのが目安。
detail['eFG%'] = (detail.FGM + 0.5 * detail.FGM3) / detail.FGA
# PPT:シュート1本あたりの得点。得点期待値？Points Per Shot
detail['FGM2'] = detail.FGM - detail.FGM3 - detail.FTM
detail['PPS'] = (2 * detail.FGM2 + 3 * detail.FGM3) / detail.FGA
# 2PPS:2Pの得点期待値。1を超えるのが目安。
detail['FGA2'] = detail.FGA - detail.FGA3 - detail.FTA
detail['2PPS'] = 2 * detail.FGA2
# 3PPS:3Pの得点期待値。1を超えるのが目安。
detail['3PPS'] = 3 * detail['3p_goal_rate']
# TOV%:ターンオーバー率。「Four Factors」の1つ。1回の攻撃でどれだけTOVするか。少ないほうがよい。
detail['TOV%'] = detail.TO / detail.POSS
# OREB%:オフェンスリバウンド率。「Four Factors」の1つ。OREBの機会にどれだけ奪えるか。OREB機会は自チームのOREBと相手のDREBの合計。
detail['OREB%'] = detail.OR / (detail.OR + detail.O_DREB)
# DREB%:ディフェンスリバウンド率。DREBの機会にどれだけ奪えるか。DREB機会は自チームのDREBと相手のOREBの合計。
detail['DREB%'] = detail.DR / (detail.DR + detail.O_OREB)
# REB%:リバウンド率。リバウンド機会にどれだけ奪えるか。
detail['REB'] = detail.DR + detail.OR
# REB = detail.DR + detail.OR
detail['REB%'] = (detail.REB) / (detail.REB + detail.O_REB)
# FTR:フリースローレーティング。「Four Factors」の1つ。フリースローをどれだけ獲得できるか。アタックできてるかどうかの指標。
detail['FTR'] = detail.FTA / detail.FGA
# TS%:True Shooting Percentage。真のシュート成功率。eFG%がフィールドゴールだけなのに対して、TS%はFTも加味した得点効率。
detail['TS%'] = 0.5 * detail.Score / (detail.FGA + 0.44 * detail.FTA)
# 3P/2P:3PAと2PAの比率。アウトサイドとインサイドのどちらに重きを置いているかを測る指標。
detail['3P/2P'] = detail.FGA3 / detail.FGA2
# AST%:アシスト率。アシストによるフィールドゴールがどれだけあるかを測る指標。
detail['AST%'] = detail.Ast / detail.FGM
# AST/TO:ASTとTOVの比率。アシストが多くターンオーバーが少ないのが優秀とみることが多い。
detail['AST/TO'] = detail.Ast / detail.TO
# PACE:ペース。試合展開の速さの目安。1試合＝40分あたりの攻撃回数。
detail['PACE'] = detail.POSS * 5 / 200
# 以下小川さんの作った特徴量
#対戦相手のPOSS
detail['O_POSS'] = detail.O_FGA + 0.44 * detail.O_FTA + detail.O_TO - detail.O_OREB
#LO_DRTG:対戦相手のディフェンスレーティング
detail['O_DRTG'] = detail.Score / detail.O_POSS * 100
#AdjEM 	自チームのオフェンス　ー　対戦相手のディフェンス
detail['AdjEM'] = detail.ORTG - detail.O_DRTG


# dt = detail.groupby(['Season','TeamID'])['FGM','FGA','FGM3','FGA3','FTM','FTA',
#                                          'OR','DR','Ast','TO','Stl','Blk','PF',
#                                           'goal_rate', '3p_goal_rate',
#                                          'ft_goal_rate', 'total_attack', 'Points_Per_Possession',
#                                         'POSS', 'ORTG', 'DRTG', 'NRTG', 'eFG%', 'FGM2', 'PPS', 'FGA2',
#                                         '2PPS', '3PPS', 'TOV%', 'OREB%', 'DREB%', 'FTR', 'TS%', '3P/2P',
#                                         'AST%', 'AST/TO', 'PACE', 'AdjEM', 'REB', 'REB%']\
#                                         .mean().fillna(0).reset_index()

dt = detail.groupby(['Season','TeamID'])['Points_Per_Possession', 'NRTG', 'PACE', 'TS%', 'AdjEM',
                                         'eFG%', 'TOV%', 'OREB%', 'DREB%', 'FTR']\
                                        .mean().fillna(0).reset_index()

# dt.drop(['FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF'], axis=1, inplace=True)
# dt = detail.groupby(['Season','TeamID'])['FGM','FGA','FGM3','FGA3','FTM','FTA',
#                                          'OR','DR','Ast','TO','Stl','Blk','PF',
#                                           'goal_rate', '3p_goal_rate',
#                                          'ft_goal_rate', 'total_attack', 'Points_Per_Possession',
#                                         'POSS', 'ORTG', 'DRTG', 'NRTG', 'eFG%', 'FGM2', 'PPS', 'FGA2',
#                                         'TOV%', 'OREB%', 'DREB%', 'FTR', 'TS%', '3P/2P',
#                                         'AST%', 'AST/TO', 'PACE', '2PPS', '3PPS']\
#                                         .mean().fillna(0).reset_index()

del w, l

<ipython-input-176-e82fde63750c>:105: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dt = detail.groupby(['Season','TeamID'])['Points_Per_Possession', 'NRTG', 'PACE', 'TS%', 'AdjEM',


In [177]:
dt

,Season,TeamID,Points_Per_Possession,NRTG,PACE,TS%,AdjEM,eFG%,TOV%,OREB%,DREB%,FTR
0,2003,1102,0.978768,0.336084,1.364071,0.606248,-0.314082,0.584407,0.207292,0.168235,0.630384,0.446693
1,2003,1103,0.987307,0.352364,1.751963,0.585812,-0.052350,0.536564,0.181540,0.305803,0.626998,0.465135
2,2003,1104,0.872026,5.874618,1.652536,0.521729,-0.188675,0.475785,0.201266,0.371256,0.686897,0.372350
3,2003,1105,0.800350,-6.586102,1.909538,0.504339,-1.480413,0.457983,0.243300,0.335166,0.641434,0.359501
4,2003,1106,0.801559,-0.391725,1.682000,0.509554,-0.593199,0.481697,0.253060,0.349480,0.679342,0.307563
...,...,...,...,...,...,...,...,...,...,...,...,...
6529,2021,1467,0.896806,-0.564522,1.706944,0.544807,1.414556,0.515241,0.190317,0.183695,0.709350,0.316375
6530,2021,1468,1.010538,6.608821,1.635778,0.591364,0.650607,0.558780,0.164687,0.240750,0.726614,0.289987
6531,2021,1469,0.827340,-13.937360,1.860737,0.506050,-0.477400,0.454704,0.205347,0.215734,0.702829,0.357629
6532,2021,1470,0.913223,-6.122339,1.648400,0.536670,-0.808112,0.485571,0.163102,0.153605,0.692172,0.395592


In [178]:
detail

,Season,Score,TeamID,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,...,REB%,FTR,TS%,3P/2P,AST%,AST/TO,PACE,O_POSS,O_DRTG,AdjEM
0,2003,68,1104,27,58,3,14,11,18,14,...,0.542857,0.310345,0.515777,0.538462,0.481481,0.565217,1.873,70.68,96.208263,-5.444782
1,2003,70,1272,26,62,8,20,10,19,15,...,0.488636,0.306452,0.497442,0.869565,0.615385,1.230769,1.709,67.80,103.244838,-0.845774
2,2003,73,1266,24,58,8,18,17,29,17,...,0.447917,0.500000,0.515828,1.636364,0.625000,1.500000,1.594,64.12,113.849033,0.642811
3,2003,56,1296,18,38,3,9,17,31,6,...,0.403226,0.815789,0.542215,-4.500000,0.611111,0.916667,1.441,57.60,97.222222,-0.067469
4,2003,77,1400,30,61,6,14,11,13,17,...,0.520000,0.213115,0.577038,0.411765,0.400000,0.857143,1.593,62.88,122.455471,-1.614291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96682,2021,79,1261,30,74,10,27,9,13,15,...,0.465909,0.175676,0.495484,0.794118,0.333333,2.000000,1.743,71.20,110.955056,2.355328
96683,2021,72,1259,29,62,5,22,9,16,12,...,0.543860,0.258065,0.521437,0.916667,0.413793,1.000000,1.726,67.16,107.206671,-2.919301
96684,2021,54,1153,18,65,8,33,10,15,14,...,0.462687,0.230769,0.377095,1.941176,0.611111,1.100000,1.690,65.08,82.974800,-3.093143
96685,2021,88,1326,29,65,8,25,22,30,8,...,0.466667,0.461538,0.562660,2.500000,0.379310,1.375000,1.955,75.08,117.208311,-4.676342


In [179]:
# 個人が集計したチームランキング
MMOrdinals = pd.read_csv(DATA_DIR + '/MMasseyOrdinals.csv')

MOR_127_128 = MMOrdinals[(MMOrdinals.SystemName == 'MOR') & \
                ((MMOrdinals.RankingDayNum == 127) \
                 | (MMOrdinals.RankingDayNum == 128))]\
                [['Season','TeamID','OrdinalRank']]
MOR_50_51 = MMOrdinals[(MMOrdinals.SystemName == 'MOR') & \
                ((MMOrdinals.RankingDayNum == 50) \
                 | (MMOrdinals.RankingDayNum == 51))]\
                [['Season','TeamID','OrdinalRank']]
MOR_15_16 = MMOrdinals[(MMOrdinals.SystemName == 'MOR') & \
                ((MMOrdinals.RankingDayNum == 15) \
                 | (MMOrdinals.RankingDayNum == 16))]\
                [['Season','TeamID','OrdinalRank']]

MOR_127_128 = MOR_127_128.rename(columns={'OrdinalRank':'OrdinalRank_127_128'})
MOR_50_51 = MOR_50_51.rename(columns={'OrdinalRank':'OrdinalRank_50_51'})
MOR_15_16 = MOR_15_16.rename(columns={'OrdinalRank':'OrdinalRank_15_16'})

MOR = MOR_127_128.merge(MOR_50_51, how='left', on=['Season','TeamID'])\
        .merge(MOR_15_16, how='left', on=['Season','TeamID'])

## normalizing Rank values by its season maxium as it varies by seasons
MOR_max = MOR.groupby('Season')['OrdinalRank_127_128','OrdinalRank_50_51',
                                'OrdinalRank_15_16'].max().reset_index()
MOR_max.columns = ['Season', 'maxRank_127_128', 'maxRank_50_51', 'maxRank_15_16']

MOR_tmp = MMOrdinals[(MMOrdinals.SystemName == 'MOR') \
                     & (MMOrdinals.RankingDayNum < 133)]
MOR_stats = MOR_tmp.groupby(['Season','TeamID'])['OrdinalRank']\
            .agg(['max','min','std','mean']).reset_index()
MOR_stats.columns = ['Season','TeamID','RankMax','RankMin','RankStd','RankMean']

MOR = MOR.merge(MOR_max, how='left', on='Season')\
        .merge(MOR_stats, how='left', on=['Season','TeamID'])
MOR['OrdinalRank_127_128'] = MOR['OrdinalRank_127_128'] / MOR['maxRank_127_128']
MOR['OrdinalRank_50_51'] = MOR['OrdinalRank_50_51'] / MOR['maxRank_50_51']
MOR['OrdinalRank_15_16'] = MOR['OrdinalRank_15_16'] / MOR['maxRank_15_16']
MOR['RankTrans_50_51_to_127_128'] = MOR['OrdinalRank_127_128'] \
                                    - MOR['OrdinalRank_50_51']
MOR['RankTrans_15_16_to_127_128'] = MOR['OrdinalRank_127_128'] \
                                    - MOR['OrdinalRank_15_16']

# MOR['RankMax'] = MOR['RankMax'] / MOR['maxRank_127_128']
# MOR['RankMin'] = MOR['RankMin'] / MOR['maxRank_127_128']
# MOR['RankStd'] = MOR['RankStd'] / MOR['maxRank_127_128']
# MOR['RankMean'] = MOR['RankMean'] / MOR['maxRank_127_128']

MOR.drop(['OrdinalRank_50_51','OrdinalRank_15_16', 'maxRank_50_51'
          ,'maxRank_15_16'],axis=1, inplace=True)

del MOR_127_128, MOR_50_51, MOR_15_16, MOR_max, MOR_tmp, MOR_stats

<ipython-input-179-9e529f19c022>:25: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  MOR_max = MOR.groupby('Season')['OrdinalRank_127_128','OrdinalRank_50_51',


In [180]:
MOR

,Season,TeamID,OrdinalRank_127_128,maxRank_127_128,RankMax,RankMin,RankStd,RankMean,RankTrans_50_51_to_127_128,RankTrans_15_16_to_127_128
0,2003,1102,0.418960,327,146,106,12.670160,127.928571,0.030581,NaN
1,2003,1103,0.431193,327,184,129,15.144723,156.142857,-0.070336,NaN
2,2003,1104,0.058104,327,41,13,8.482691,22.571429,0.012232,NaN
3,2003,1105,0.948012,327,314,262,17.052762,296.785714,0.097859,NaN
4,2003,1106,0.896024,327,293,242,11.841675,268.928571,0.088685,NaN
...,...,...,...,...,...,...,...,...,...,...
6528,2021,1467,0.717579,347,342,213,45.949766,288.333333,-0.223597,NaN
6529,2021,1468,0.412104,347,354,101,67.183119,176.000000,-0.187336,NaN
6530,2021,1469,0.919308,347,356,267,22.734073,306.466667,0.120989,NaN
6531,2021,1470,0.556196,347,357,147,51.558381,184.533333,0.007176,NaN


In [181]:
wl_1 = wl.loc[:,['Season','TeamID','win_pct_A','win_pct_N',
                 'win_pct_H','win_pct_All']]
wl_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                else str(col) for col in wl_1.columns ]

wl_2 = wl.loc[:,['Season','TeamID','win_pct_A','win_pct_N',
                 'win_pct_H','win_pct_All']]
wl_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                else str(col) for col in wl_2.columns ]

scr_1 = scr.copy()
scr_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                 else str(col) for col in scr_1.columns ]

scr_2 = scr.copy()
scr_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                 else str(col) for col in scr_2.columns ]

dt_1 = dt.copy()
dt_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                else str(col) for col in dt_1.columns ]

dt_2 = dt.copy()
dt_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                else str(col) for col in dt_2.columns ]

MOR_1 = MOR.copy()
MOR_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                 else str(col) for col in MOR_1.columns ]

MOR_2 = MOR.copy()
MOR_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                 else str(col) for col in MOR_2.columns ]

q_seed_1 = q_seed.copy()
q_seed_1.columns = [str(col) + '_1' if col not in ['Season', 'TeamID'] \
                   else str(col) for col in q_seed_1.columns]

q_seed_2 = q_seed.copy()
q_seed_2.columns = [str(col) + '_2' if col not in ['Season', 'TeamID'] \
                   else str(col) for col in q_seed_2.columns]

In [182]:
# TCResults = pd.read_csv(DATA_DIR + '/MNCAATourneyCompactResults.csv')

# tourney1 = TCResults.loc[:, ['Season','WTeamID','LTeamID']]
# tourney1.columns = ['Season','TeamID1','TeamID2']
# tourney1['result'] = 1

# tourney2 = TCResults.loc[:, ['Season','LTeamID','WTeamID']]
# tourney2.columns = ['Season','TeamID1','TeamID2']
# tourney2['result'] = 0

# tourney = pd.concat([tourney1, tourney2])
# del tourney1, tourney2

In [183]:
regular_results['T1_TeamID'] = regular_results['T1_TeamID'].astype(np.int64)
regular_results['T2_TeamID'] = regular_results['T2_TeamID'].astype(np.int64)
tourney1 = regular_results.loc[:, ['Season','T1_TeamID','T2_TeamID', 'T1_win']]
tourney1.columns = ['Season','TeamID1','TeamID2', 'result']

tourney2 = regular_results.loc[:, ['Season','T2_TeamID','T1_TeamID', 'T2_win']]
tourney2.columns = ['Season','TeamID1','TeamID2', 'result']

regular_final = pd.concat([tourney1, tourney2])
del tourney1, tourney2
regular_final

,Season,TeamID1,TeamID2,result
0,1985,1228,1328,1
1,1985,1328,1228,0
2,1985,1354,1106,0
3,1985,1106,1354,1
4,1985,1112,1223,1
...,...,...,...,...
341465,2021,1261,1104,0
341466,2021,1222,1153,1
341467,2021,1153,1222,0
341468,2021,1326,1228,0


In [184]:
def merge_data(df):

#     df = df.merge(wl_1, how='left', left_on=['Season','TeamID1'],
#                   right_on=['Season','TeamID'])
#     df = df.merge(wl_2, how='left', left_on=['Season','TeamID2'],
#                   right_on=['Season','TeamID'])
#     df = df.drop(['TeamID_x','TeamID_y'], axis=1)


#     df = df.merge(scr_1, how='left', left_on=['Season','TeamID1'],
#                   right_on=['Season','TeamID'])
#     df = df.merge(scr_2, how='left', left_on=['Season','TeamID2'],
#                   right_on=['Season','TeamID'])
#     df = df.drop(['TeamID_x','TeamID_y'], axis=1)

#     df['win_pct_A_diff'] = df['win_pct_A_1'] - df['win_pct_A_2']
#     df['win_pct_N_diff'] = df['win_pct_N_1'] - df['win_pct_N_2']
#     df['win_pct_H_diff'] = df['win_pct_H_1'] - df['win_pct_H_2']
#     df['win_pct_All_diff'] = df['win_pct_All_1'] - df['win_pct_All_2']

#     df['Score_A_diff'] = df['Score_A_1'] - df['Score_A_2']
#     df['Score_N_diff'] = df['Score_N_1'] - df['Score_N_2']
#     df['Score_H_diff'] = df['Score_H_1'] - df['Score_H_2']
#     df['Score_All_diff'] = df['Score_All_1'] - df['Score_All_2']

#     df['relScore_A_diff'] = df['relScore_A_1'] - df['relScore_A_2']
#     df['relScore_N_diff'] = df['relScore_N_1'] - df['relScore_N_2']
#     df['relScore_H_diff'] = df['relScore_H_1'] - df['relScore_H_2']
#     df['relScore_All_diff'] = df['relScore_All_1'] - df['relScore_All_2']

    df = df.merge(dt_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(dt_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)

    df = df.merge(MOR_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(MOR_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)
    
    df = df.merge(q_seed_1, how='left', left_on=['Season', 'TeamID1'],
                 right_on=['Season', 'TeamID'])
    df = df.merge(q_seed_2, how='left', left_on=['Season', 'TeamID2'],
                 right_on=['Season', 'TeamID'])
    df = df.drop(['TeamID_x', 'TeamID_y'], axis=1)

#     df['OrdinalRank_127_128_diff'] = df['OrdinalRank_127_128_1'] \
#         - df['OrdinalRank_127_128_2']

    df = df.fillna(-1)
    
    for col in df.columns:
        if (df[col] == np.inf).any() or (df[col] == -np.inf).any():
            df[col][(df[col] == np.inf) | (df[col] == -np.inf)] = -1
    
    return df

# tourney = merge_data(tourney)
final_regular_result = merge_data(regular_final)
final_regular_result = final_regular_result.loc[final_regular_result.Season >= 2015,:].reset_index(drop=True)

if STAGE_1:
    final_regular_result = final_regular_result.loc[final_regular_result.Season < 2022, :]

In [185]:
if STAGE_1:
    MSampleSubmission = pd.read_csv(DATA_DIR + '/MSampleSubmissionStage2.csv')
else:
    MSampleSubmission = pd.read_csv(DATA_DIR + None) # put stage 2 submission file link here

test1 = MSampleSubmission.copy()
test1['Season'] = test1.ID.apply(lambda x: int(x[0:4]))
test1['TeamID1'] = test1.ID.apply(lambda x: int(x[5:9]))
test1['TeamID2'] = test1.ID.apply(lambda x: int(x[10:14]))

test2 = MSampleSubmission.copy()
test2['Season'] = test2.ID.apply(lambda x: int(x[0:4]))
test2['TeamID1'] = test2.ID.apply(lambda x: int(x[10:14]))
test2['TeamID2'] = test2.ID.apply(lambda x: int(x[5:9]))

test = pd.concat([test1,test2]).drop(['Pred'], axis=1)
test = merge_data(test)

In [186]:
final_regular_result.to_csv('final_regular_result.csv', index=False)
test.to_csv('stage1_test.csv', index=False)

In [187]:
final_regular_result

,Season,TeamID1,TeamID2,result,Points_Per_Possession_1,NRTG_1,PACE_1,TS%_1,AdjEM_1,eFG%_1,...,RankStd_2,RankMean_2,RankTrans_50_51_to_127_128_2,RankTrans_15_16_to_127_128_2,seed_1,division_1,quality_1,seed_2,division_2,quality_2
0,2015,1103,1420,1,0.888400,6.413600,1.608088,0.526925,-0.654677,0.496626,...,11.483364,321.352941,0.062678,0.091168,-1.0,-1,-1.000000,-1.0,-1,-1.0
1,2015,1420,1103,0,0.791964,-17.094305,1.600133,0.502975,0.287299,0.466930,...,6.456301,107.058824,-0.056980,-0.019943,-1.0,-1,-1.000000,-1.0,-1,-1.0
2,2015,1104,1406,1,0.926717,3.369313,1.564742,0.556632,0.187854,0.505980,...,30.021928,229.235294,0.128205,0.182336,-1.0,-1,-1.000000,-1.0,-1,-1.0
3,2015,1406,1104,0,0.826042,-5.631398,1.536839,0.501806,-0.502381,0.450531,...,7.406774,52.117647,-0.005698,0.025641,-1.0,-1,-1.000000,-1.0,-1,-1.0
4,2015,1112,1291,1,0.992801,26.530881,1.656294,0.578984,0.133540,0.532952,...,21.371538,240.352941,-0.065527,0.011396,2.0,Z,691.305669,-1.0,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144671,2021,1261,1104,0,0.994681,9.775507,1.826444,0.574367,0.568291,0.532948,...,22.089644,25.666667,-0.173185,-1.000000,-1.0,-1,-1.000000,-1.0,-1,-1.0
144672,2021,1222,1153,1,0.968277,27.172667,1.658500,0.552980,1.250096,0.510378,...,20.113488,93.133333,0.001582,-1.000000,-1.0,-1,-1.000000,-1.0,-1,-1.0
144673,2021,1153,1222,0,0.860595,-4.501280,1.797261,0.518065,-1.255999,0.484938,...,2.693908,9.600000,0.000888,-1.000000,-1.0,-1,-1.000000,-1.0,-1,-1.0
144674,2021,1326,1228,0,1.010755,9.237440,1.709367,0.579279,0.709214,0.532412,...,5.942582,7.200000,-0.016565,-1.000000,-1.0,-1,-1.000000,-1.0,-1,-1.0


In [188]:
test

,ID,Season,TeamID1,TeamID2,Points_Per_Possession_1,NRTG_1,PACE_1,TS%_1,AdjEM_1,eFG%_1,...,RankStd_2,RankMean_2,RankTrans_50_51_to_127_128_2,RankTrans_15_16_to_127_128_2,seed_1,division_1,quality_1,seed_2,division_2,quality_2
0,2021_1101_1104,2021,1101,1104,0.939144,19.965877,1.792696,0.560750,-0.405907,0.528451,...,22.089644,25.666667,-0.173185,-1.0,-1.0,-1,-1.0,-1.0,-1,-1.0
1,2021_1101_1111,2021,1101,1111,0.939144,19.965877,1.792696,0.560750,-0.405907,0.528451,...,41.363315,212.066667,0.177100,-1.0,-1.0,-1,-1.0,-1.0,-1,-1.0
2,2021_1101_1116,2021,1101,1116,0.939144,19.965877,1.792696,0.560750,-0.405907,0.528451,...,18.609521,18.200000,-0.041452,-1.0,-1.0,-1,-1.0,-1.0,-1,-1.0
3,2021_1101_1124,2021,1101,1124,0.939144,19.965877,1.792696,0.560750,-0.405907,0.528451,...,0.833809,1.866667,0.000161,-1.0,-1.0,-1,-1.0,-1.0,-1,-1.0
4,2021_1101_1140,2021,1101,1140,0.939144,19.965877,1.792696,0.560750,-0.405907,0.528451,...,11.909580,52.866667,-0.044955,-1.0,-1.0,-1,-1.0,-1.0,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4551,2021_1452_1457,2021,1457,1452,0.948061,17.402617,1.824667,0.566447,0.718406,0.531264,...,9.255629,34.333333,0.002745,-1.0,-1.0,-1,-1.0,-1.0,-1,-1.0
4552,2021_1452_1458,2021,1458,1452,0.948294,8.072436,1.676897,0.535697,-0.254226,0.496072,...,9.255629,34.333333,0.002745,-1.0,-1.0,-1,-1.0,-1.0,-1,-1.0
4553,2021_1455_1457,2021,1457,1455,0.948061,17.402617,1.824667,0.566447,0.718406,0.531264,...,18.531055,85.400000,-0.069899,-1.0,-1.0,-1,-1.0,-1.0,-1,-1.0
4554,2021_1455_1458,2021,1458,1455,0.948294,8.072436,1.676897,0.535697,-0.254226,0.496072,...,18.531055,85.400000,-0.069899,-1.0,-1.0,-1,-1.0,-1.0,-1,-1.0
